In [41]:
using Pkg
Pkg.add("PyCall")
using PyCall

   Resolving package versions...
  No Changes to `C:\Users\dafne\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\dafne\.julia\environments\v1.11\Manifest.toml`


In [42]:
sp = pyimport_conda("sympy", "sympy")
np = pyimport_conda("numpy", "numpy")
sp_o = pyimport_conda("scipy.optimize", "scipy")
sp_i = pyimport_conda("scipy.integrate", "scipy")
pg = pyimport_conda("pygad", "pygad")
plt = pyimport_conda("matplotlib.pyplot", "matplotlib")

PyObject <module 'matplotlib.pyplot' from 'C:\\Users\\dafne\\.julia\\conda\\3\\x86_64\\Lib\\site-packages\\matplotlib\\pyplot.py'>

In [43]:
target_ns = 0.965
target_r = 0.0266
target_N = 60

println(target_ns)
println(target_r)
println(target_N)

0.965
0.0266
60


In [44]:
function polynomial(coefficients)
    """computing inflationary potential potential
    return parameters : Number of e-folds(N), spectral index (ns), tensor-to-scalar ratio (r)"""
    phi = sp.Symbol("phi")
    c0, c1, c2, c3, c4 = coefficients

    # inflationary potential skeleton which takes the form of a polynomial.
    V_phi_expr = c0 + c1*phi + (c2*phi^2) + (c3*phi^3) + (c4*phi^4)
    V_first = sp.diff(V_phi_expr, phi) # computing derivatives.
    V_second = sp.diff(V_first, phi)
    
    # converting to numerical functions.
    V = sp.lambdify(phi, V_phi_expr, "numpy")
    V_1 = sp.lambdify(phi, V_first, "numpy")
    V_2 = sp.lambdify(phi, V_second, "numpy")

    function epsilon(phi_val)
        """computing first slow-roll parameter"""
        v = V(phi_val)
        v1 = V_1(phi_val)

        # if the input is not valid.
        if v <= 0 || np.isnan(v) || np.isnan(v1)
            return 1e6
        end
        if np.abs(v1) < 1e-15 # to avoid division by 0
                return 1e6
        end
        epsilon_calculation = (1/2) * (v1/v)^2
        return min(epsilon_calculation, 1000)
    end
                
    function eta(phi_val)
        """computing second slow-roll parameter"""
        v = V(phi_val)
        v2 = V_2(phi_val)

        #if the input is not valid.
        if v <= 0 || np.isnan(v) || np.isnan(v1)
            return 1e6
        end
        eta_calculation = v2/v
        return min(eta_calculation, 1000)
    end

    # finding scalar field range
    phi_start = 20 # started at a higher value to widen the range of the scalar field.
    phi_end = 0.01 # starting from the smallest value closer to 0, to find a global minimum.

    # checking if evalution is taking place at these points distinctively.
    epsilon_start = epsilon(phi_start)
    epsilon_end = epsilon(phi_end)

    if epsilon_start >= 1e6 || epsilon_end >= 1e6
        return nothing, nothing, nothing
    end
                            
    # calulcating the number of e-folds.
    function integrand(x)
            v = V(x)
            v1 = V_1(x)
            if np.abs(v1) < 1e-15 || v <= 0
                return 0.0
        end
            return v/v1
    end

    # applying error handling to the integrand.
    phi_points = range(phi_end, phi_start; phi_length, length = 100)
    integrand_values = [integrand(x) for x in phi_points]
    step_size = step(phi_points)
    N_value = sum(step_size * (integrand_values[i] + integrand_values[i+1]) / 2 for i in 1:length(integrand_values)-1)

    # calculating observables at horizon exit.
    epsilon_star = epsilon(phi_start)
    eta_star = eta(phi_star)

    if eta_star >= 1e6 || epsilon_star >= 1e6
        return nothing, nothing, nothing
    end

    # computing observables.
    ns_computed_value = 1 - (6*epsilon_star) + (2*eta_star)
    r_computed_value  = 16*epsilon_star

    return ns_computed_value, r_computed_value, N_value
end
               

polynomial (generic function with 1 method)

In [45]:
function fitness(ga, solution, idx)
    """fitness function"""

    # evaluating the model.
    result = polynomial(solution)

    if result == nothing || result[1] == nothing 
        return 1e-10
    end

    ns_computed_value, r_computed_value, N_value = result
    
    if !(N_value <=60)
        return 1e-10
    end
    if !(0.9 <= ns_computed_value <= 1)
        return 1e-10
    end
    if r_computed_value < 0 || r_computed_value > 0.6
        return 1e-10
    end

    # computing absolute value.
    ns_abs = abs(ns_computed_value - target_ns) 
    r_abs = abs(r_computed_value - target_r) 
    N_abs = abs(N_value - target_N) 
    
    # weighted error calculation.
    ns_error = ns_abs / 0.1
    r_error = r_abs /0.01
    N_error = N_abs / 10 # scaling each value differently depending on which range they lie in.

    # penalizing values heavily.
    if ns_abs > 0.5
        ns_error *= 10
    end
    if r_abs > 0.5
        r_error *= 10
    end
    if N_abs > 40
        N_error *=10
    end

    # fitness equation.
    F = (ns_abs)^2 + (r_abs)^2 + ((1 / np.pi) * atan(N_value - 60)) + 1/2

    # loss equation (l = (f)^-1, inverse of the fitness)
    L = 1 / F

    return F, L
end 

fitness (generic function with 1 method)

In [47]:
"""gentic algorithm code using PYGAD"""
ga = pg.GA(
    num_generations = 500, # max. number of generations GA will run for.
    sol_per_pop = 100, # number of chromosomes in each population.
    num_parents_mating = 40, # number of parents which are selected in each generation to mate.
    num_genes = 5,
    fitness_func = fitness,
    init_range_low = -1.0, # initial value for the genes starts wihtin this rnage (completely random).
    init_range_high = 1.0,
    mutation_percent_genes = 52.1,# 52.1% of the genes in each offspring will be mutated.
    mutation_by_replacement = true,
    random_mutation_min_val = -1.0,
    random_mutation_max_val = 1.0,
    crossover_type = "single_point",
    mutation_type = "random",
    parent_selection_type = "tournament",
    K_tournament = 3,
    keep_parents = 5, # after mating, keeping only 5 parents.
    save_best_solutions = true,
    stop_criteria = ["saturate_10"]  # stop if no improvement for 10 generations
)  

ga.run() # running genetic algortihm.

solution, F, idx = ga.best_solution() # obtaining best solution.
println(f"\nBest Coefficients", solution)
@printf("Fitness Score: %.2f\n", F) 
    

'PyCall.jlwrap' object has no attribute '__code__'
Traceback (most recent call last):
  File "C:\Users\dafne\.julia\conda\3\x86_64\Lib\site-packages\pygad\pygad.py", line 956, in __init__
    if fitness_func.__code__.co_argcount == 3:
       ^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'PyCall.jlwrap' object has no attribute '__code__'. Did you mean: '__call__'?


LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\dafne\.julia\packages\PyCall\1gn3u\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("'PyCall.jlwrap' object has no attribute '__code__'")
  File "C:\Users\dafne\.julia\conda\3\x86_64\Lib\site-packages\pygad\pygad.py", line 1339, in __init__
    raise e
  File "C:\Users\dafne\.julia\conda\3\x86_64\Lib\site-packages\pygad\pygad.py", line 956, in __init__
    if fitness_func.__code__.co_argcount == 3:
       ^^^^^^^^^^^^^^^^^^^^^


In [48]:
""""error testing to obtain best solution"""
result = polynomial(solution)

if result[1] !== nothing
    ns_best, r_best, N_best = result

    println("\nFinal Results:")
    @printf("n_s: %.4f (target: %.4f)\n", ns_best, target_ns)
    @printf("r: %.4f (target: %.4f)\n", r_best, target_r)
    @printf("N: %.1f (target: %.1f)\n", N_best, target_N)

    println("\nErrors:")
    @printf("Δn_s: %.4f\n", abs(ns_best - target_ns))
    @printf("Δr: %.4f\n", abs(r_best - target_r))
    @printf("ΔN: %.1f\n", abs(N_best - target_N))

    # Check if the solution is good
    if abs(ns_best - target_ns) < 0.1 &&
       abs(r_best - target_r) < 0.1 &&
       abs(N_best - target_N) < 10
        println("YASSSSS")
    else
        println("sad face :(")
        println("- Running for more generations")
        println("- Trying different target values")
        println("- Using a different potential form")
    end
else
    println("\nERROR: Best solution failed to evaluate!")
    println("The optimization may need different constraints or initialization.")
end


LoadError: UndefVarError: `solution` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The last two cells are written perfectly fine in Julia notation but the only reason they do not work is due to the PYGAD library being a pythong library which does not accept the fitness function  written in Julia. This will be a bit tricky since the fitness function that I have written in both python and Julia includes the loss and fitness since L = F^-1, but this can be fixed by seperating the two and just calling them inside the second function. 
SO, PySR will have to contain a julia written LOSS and PyGAD will have to contain a python written FITNESS.